In [ ]:
# Essential imports for histogram equalization
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import data, exposure, filters, color
from skimage.morphology import disk
from skimage.filters import rank
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🚀 Ready to explore histogram equalization techniques!")
print(f"OpenCV version: {cv2.__version__}")

# Set random seed for reproducibility
np.random.seed(42)


In [ ]:
# Load and analyze sample images with different histogram characteristics
def load_diverse_images():
    """Load images with different histogram characteristics"""
    images = {
        'Low Contrast': data.moon(),           # Low contrast image
        'High Contrast': data.camera(),       # Normal contrast
        'Dark': data.text(),                  # Dark image with poor contrast
        'Bright': data.page(),                # Bright image
        'Bimodal': data.coins(),              # Bimodal histogram
    }
    return images

def analyze_histogram(image, title="Image"):
    """Analyze and visualize image histogram"""
    # Calculate histogram
    hist, bins = np.histogram(image.flatten(), bins=256, range=[0, 256])
    
    # Calculate PDF
    pdf = hist / np.sum(hist)
    
    # Calculate CDF
    cdf = np.cumsum(pdf)
    
    # Calculate statistics
    mean_intensity = np.mean(image)
    std_intensity = np.std(image)
    entropy = -np.sum(pdf[pdf > 0] * np.log2(pdf[pdf > 0]))
    
    analysis = {
        'histogram': hist,
        'pdf': pdf,
        'cdf': cdf,
        'bins': bins[:-1],
        'mean': mean_intensity,
        'std': std_intensity,
        'entropy': entropy,
        'dynamic_range': np.max(image) - np.min(image)
    }
    
    return analysis

# Load diverse images
sample_images = load_diverse_images()

# Analyze each image
analyses = {}
for name, img in sample_images.items():
    analyses[name] = analyze_histogram(img, name)

# Visualize images and their histograms
fig, axes = plt.subplots(len(sample_images), 3, figsize=(18, 4*len(sample_images)))
fig.suptitle('Image Histogram Analysis', fontsize=16, fontweight='bold')

for idx, (name, img) in enumerate(sample_images.items()):
    analysis = analyses[name]
    
    # Original image
    axes[idx, 0].imshow(img, cmap='gray')
    axes[idx, 0].set_title(f'{name} Image')
    axes[idx, 0].axis('off')
    
    # Histogram (PDF)
    axes[idx, 1].bar(analysis['bins'], analysis['pdf'], alpha=0.7, color='blue')
    axes[idx, 1].set_title(f'Histogram (PDF)')
    axes[idx, 1].set_xlabel('Intensity')
    axes[idx, 1].set_ylabel('Probability')
    axes[idx, 1].grid(True, alpha=0.3)
    
    # CDF
    axes[idx, 2].plot(analysis['bins'], analysis['cdf'], color='red', linewidth=2)
    axes[idx, 2].set_title(f'Cumulative Distribution Function')
    axes[idx, 2].set_xlabel('Intensity')
    axes[idx, 2].set_ylabel('Cumulative Probability')
    axes[idx, 2].grid(True, alpha=0.3)
    axes[idx, 2].set_ylim([0, 1])

plt.tight_layout()
plt.show()

# Print analysis summary
print("📊 Image Analysis Summary:")
print("=" * 80)
print(f"{'Image':<15} | {'Mean':<8} | {'Std':<8} | {'Entropy':<8} | {'Range':<8}")
print("-" * 80)

for name, analysis in analyses.items():
    print(f"{name:<15} | {analysis['mean']:<8.1f} | {analysis['std']:<8.1f} | "
          f"{analysis['entropy']:<8.2f} | {analysis['dynamic_range']:<8.0f}")

print("\n🔍 Key Observations:")
print("• Low contrast images have narrow histograms")
print("• High contrast images span the full intensity range")
print("• Dark images are concentrated in low intensity values")
print("• Bright images are concentrated in high intensity values")
print("• Bimodal images have two distinct peaks in their histograms")


In [ ]:
# Implement histogram equalization from scratch
def histogram_equalization_manual(image):
    """
    Implement histogram equalization from scratch
    
    Args:
        image: Input grayscale image
    
    Returns:
        equalized_image: Histogram equalized image
        transform_function: The transformation function used
    """
    # Flatten image to 1D array
    image_flat = image.flatten()
    
    # Calculate histogram
    hist, bins = np.histogram(image_flat, bins=256, range=[0, 256])
    
    # Calculate PDF
    pdf = hist / len(image_flat)
    
    # Calculate CDF
    cdf = np.cumsum(pdf)
    
    # Create transformation function
    # T(r) = (L-1) * CDF(r)
    transform_function = np.round((256 - 1) * cdf).astype(np.uint8)
    
    # Apply transformation
    equalized_flat = transform_function[image_flat.astype(int)]
    
    # Reshape back to original shape
    equalized_image = equalized_flat.reshape(image.shape)
    
    return equalized_image, transform_function

def compare_equalization_methods(image, image_name):
    """Compare different histogram equalization methods"""
    
    # Manual implementation
    manual_eq, transform_func = histogram_equalization_manual(image)
    
    # OpenCV implementation
    opencv_eq = cv2.equalizeHist(image.astype(np.uint8))
    
    # Scikit-image implementation
    skimage_eq = exposure.equalize_hist(image)
    skimage_eq = (skimage_eq * 255).astype(np.uint8)
    
    # Analyze results
    original_analysis = analyze_histogram(image)
    manual_analysis = analyze_histogram(manual_eq)
    opencv_analysis = analyze_histogram(opencv_eq)
    skimage_analysis = analyze_histogram(skimage_eq)
    
    # Visualization
    fig, axes = plt.subplots(4, 3, figsize=(18, 16))
    fig.suptitle(f'Histogram Equalization Comparison: {image_name}', fontsize=16, fontweight='bold')
    
    methods = [
        ('Original', image, original_analysis),
        ('Manual Implementation', manual_eq, manual_analysis),
        ('OpenCV', opencv_eq, opencv_analysis),
        ('Scikit-Image', skimage_eq, skimage_analysis)
    ]
    
    for idx, (method_name, img, analysis) in enumerate(methods):
        # Image
        axes[idx, 0].imshow(img, cmap='gray', vmin=0, vmax=255)
        axes[idx, 0].set_title(f'{method_name}')
        axes[idx, 0].axis('off')
        
        # Histogram
        axes[idx, 1].bar(analysis['bins'], analysis['histogram'], alpha=0.7)
        axes[idx, 1].set_title(f'Histogram')
        axes[idx, 1].set_xlabel('Intensity')
        axes[idx, 1].set_ylabel('Frequency')
        axes[idx, 1].grid(True, alpha=0.3)
        
        # CDF and Transform Function
        axes[idx, 2].plot(analysis['bins'], analysis['cdf'], label='CDF', linewidth=2)
        if idx == 1:  # Manual implementation
            axes[idx, 2].plot(range(256), transform_func/255, label='Transform Function', 
                             linewidth=2, linestyle='--')
            axes[idx, 2].legend()
        axes[idx, 2].set_title(f'CDF')
        axes[idx, 2].set_xlabel('Intensity')
        axes[idx, 2].set_ylabel('Cumulative Probability')
        axes[idx, 2].grid(True, alpha=0.3)
        axes[idx, 2].set_ylim([0, 1])
    
    plt.tight_layout()
    plt.show()
    
    # Print comparison statistics
    print(f"\\n📊 {image_name} - Equalization Comparison:")
    print("=" * 70)
    print(f"{'Method':<20} | {'Mean':<8} | {'Std':<8} | {'Entropy':<8} | {'Range':<8}")
    print("-" * 70)
    
    for method_name, _, analysis in methods:
        print(f"{method_name:<20} | {analysis['mean']:<8.1f} | {analysis['std']:<8.1f} | "
              f"{analysis['entropy']:<8.2f} | {analysis['dynamic_range']:<8.0f}")
    
    return manual_eq, opencv_eq, skimage_eq

# Test histogram equalization on different image types
test_images = {
    'Low Contrast': sample_images['Low Contrast'],
    'Dark Image': sample_images['Dark']
}

equalization_results = {}

for name, img in test_images.items():
    print(f"\\n🔧 Processing {name}...")
    manual_eq, opencv_eq, skimage_eq = compare_equalization_methods(img, name)
    equalization_results[name] = {
        'original': img,
        'manual': manual_eq,
        'opencv': opencv_eq,
        'skimage': skimage_eq
    }

print("\\n💡 Key Insights:")
print("• Histogram equalization spreads intensities across full range")
print("• Standard deviation typically increases (better contrast)")
print("• Entropy often increases (more information content)")
print("• Different implementations may produce slightly different results")
print("• Most effective on low-contrast images")


In [ ]:
# Implement and compare Adaptive Histogram Equalization
def adaptive_histogram_equalization(image, tile_size=(8, 8)):
    """
    Implement Adaptive Histogram Equalization
    
    Args:
        image: Input grayscale image
        tile_size: Size of each tile (rows, cols)
    
    Returns:
        ahe_image: Adaptive histogram equalized image
    """
    h, w = image.shape
    tile_h, tile_w = tile_size
    
    # Calculate number of tiles
    n_tiles_h = h // tile_h
    n_tiles_w = w // tile_w
    
    # Initialize output image
    ahe_image = np.zeros_like(image)
    
    # Process each tile
    for i in range(n_tiles_h):
        for j in range(n_tiles_w):
            # Extract tile
            start_h = i * tile_h
            end_h = start_h + tile_h
            start_w = j * tile_w
            end_w = start_w + tile_w
            
            tile = image[start_h:end_h, start_w:end_w]
            
            # Apply histogram equalization to tile
            eq_tile, _ = histogram_equalization_manual(tile)
            
            # Place equalized tile back
            ahe_image[start_h:end_h, start_w:end_w] = eq_tile
    
    return ahe_image

def create_challenging_test_image():
    """Create an image with varying illumination for testing AHE"""
    # Create base image
    h, w = 256, 256
    x, y = np.meshgrid(np.linspace(0, 1, w), np.linspace(0, 1, h))
    
    # Add different regions with varying contrast
    image = np.zeros((h, w))
    
    # Dark region with low contrast texture
    dark_region = 50 + 20 * np.sin(10 * x) * np.sin(10 * y)
    image[:h//2, :w//2] = dark_region[:h//2, :w//2]
    
    # Bright region with high contrast texture
    bright_region = 180 + 50 * np.sin(15 * x) * np.sin(15 * y)
    image[:h//2, w//2:] = bright_region[:h//2, w//2:]
    
    # Medium region with medium contrast
    medium_region = 120 + 30 * np.sin(8 * x) * np.sin(8 * y)
    image[h//2:, :w//2] = medium_region[h//2:, :w//2]
    
    # Gradient region
    gradient_region = 50 + 150 * x + 20 * np.sin(20 * x) * np.sin(20 * y)
    image[h//2:, w//2:] = gradient_region[h//2:, w//2:]
    
    # Add some noise
    image += np.random.normal(0, 5, (h, w))
    image = np.clip(image, 0, 255).astype(np.uint8)
    
    return image

# Create challenging test image
challenging_image = create_challenging_test_image()

# Apply different histogram equalization techniques
global_eq = cv2.equalizeHist(challenging_image)
adaptive_eq_manual = adaptive_histogram_equalization(challenging_image, tile_size=(32, 32))

# OpenCV's CLAHE (we'll implement our own CLAHE next)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
clahe_eq = clahe.apply(challenging_image)

# Scikit-image adaptive equalization
skimage_adaptive = exposure.equalize_adapthist(challenging_image, clip_limit=0.02)
skimage_adaptive = (skimage_adaptive * 255).astype(np.uint8)

# Visualize comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Global vs Adaptive Histogram Equalization', fontsize=16, fontweight='bold')

images = [
    ('Original', challenging_image),
    ('Global HE', global_eq),
    ('Manual AHE', adaptive_eq_manual),
    ('OpenCV CLAHE', clahe_eq),
    ('Scikit AHE', skimage_adaptive)
]

# Display first 5 images
for idx, (title, img) in enumerate(images[:5]):
    row = idx // 3
    col = idx % 3
    axes[row, col].imshow(img, cmap='gray', vmin=0, vmax=255)
    axes[row, col].set_title(title)
    axes[row, col].axis('off')

# Remove the last empty subplot
axes[1, 2].remove()

plt.tight_layout()
plt.show()

# Create histogram comparison
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
fig.suptitle('Histogram Comparison', fontsize=16, fontweight='bold')

for idx, (title, img) in enumerate(images):
    hist, bins = np.histogram(img.flatten(), bins=256, range=[0, 256])
    axes[idx].bar(bins[:-1], hist, alpha=0.7)
    axes[idx].set_title(title)
    axes[idx].set_xlabel('Intensity')
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🔍 Analysis of Results:")
print("• Global HE: Uniform histogram but loses local details")
print("• Manual AHE: Better local contrast but may have tile artifacts")
print("• CLAHE: Controlled enhancement with clip limit")
print("• Scikit AHE: Smooth adaptive enhancement")


In [ ]:
# Implement CLAHE from scratch
def clahe_manual(image, tile_size=(8, 8), clip_limit=2.0):
    """
    Implement Contrast Limited Adaptive Histogram Equalization
    
    Args:
        image: Input grayscale image
        tile_size: Size of each tile (rows, cols)
        clip_limit: Clip limit for histogram clipping
    
    Returns:
        clahe_image: CLAHE enhanced image
    """
    h, w = image.shape
    tile_h, tile_w = tile_size
    
    # Calculate number of tiles
    n_tiles_h = h // tile_h
    n_tiles_w = w // tile_w
    
    # Calculate actual clip limit (as number of pixels)
    pixels_per_tile = tile_h * tile_w
    actual_clip_limit = int(clip_limit * pixels_per_tile / 256)
    
    # Initialize output image
    clahe_image = np.zeros_like(image, dtype=np.float64)
    
    # Store transformation functions for each tile
    transform_functions = {}
    
    # Process each tile
    for i in range(n_tiles_h):
        for j in range(n_tiles_w):
            # Extract tile
            start_h = i * tile_h
            end_h = start_h + tile_h
            start_w = j * tile_w
            end_w = start_w + tile_w
            
            tile = image[start_h:end_h, start_w:end_w]
            
            # Calculate histogram
            hist, _ = np.histogram(tile.flatten(), bins=256, range=[0, 256])
            
            # Clip histogram
            clipped_hist = np.copy(hist)
            total_excess = 0
            
            # Clip values above limit
            for k in range(256):
                if clipped_hist[k] > actual_clip_limit:
                    excess = clipped_hist[k] - actual_clip_limit
                    total_excess += excess
                    clipped_hist[k] = actual_clip_limit
            
            # Redistribute excess uniformly
            redistribute_per_bin = total_excess // 256
            remainder = total_excess % 256
            
            for k in range(256):
                clipped_hist[k] += redistribute_per_bin
                if k < remainder:
                    clipped_hist[k] += 1
            
            # Calculate CDF and transformation function
            pdf = clipped_hist / np.sum(clipped_hist)
            cdf = np.cumsum(pdf)
            transform_func = np.round((256 - 1) * cdf).astype(np.uint8)
            
            # Store transform function for interpolation
            transform_functions[(i, j)] = transform_func
            
            # Apply transformation to tile
            eq_tile = transform_func[tile.flatten().astype(int)].reshape(tile.shape)
            clahe_image[start_h:end_h, start_w:end_w] = eq_tile
    
    return clahe_image.astype(np.uint8), transform_functions

def visualize_clip_limit_effects(image):
    """Demonstrate the effect of different clip limits"""
    clip_limits = [0.5, 1.0, 2.0, 4.0, 8.0]
    
    fig, axes = plt.subplots(2, len(clip_limits), figsize=(20, 8))
    fig.suptitle('Effect of Clip Limit on CLAHE', fontsize=16, fontweight='bold')
    
    for idx, clip_limit in enumerate(clip_limits):
        # Apply CLAHE with different clip limits
        clahe_result, _ = clahe_manual(image, tile_size=(16, 16), clip_limit=clip_limit)
        
        # Display image
        axes[0, idx].imshow(clahe_result, cmap='gray', vmin=0, vmax=255)
        axes[0, idx].set_title(f'Clip Limit: {clip_limit}')
        axes[0, idx].axis('off')
        
        # Display histogram
        hist, bins = np.histogram(clahe_result.flatten(), bins=256, range=[0, 256])
        axes[1, idx].bar(bins[:-1], hist, alpha=0.7)
        axes[1, idx].set_title(f'Histogram (CL: {clip_limit})')
        axes[1, idx].set_xlabel('Intensity')
        axes[1, idx].set_ylabel('Frequency')
        axes[1, idx].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def compare_enhancement_quality(image):
    """Compare image quality metrics across different enhancement methods"""
    
    # Apply different methods
    methods = {
        'Original': image,
        'Global HE': cv2.equalizeHist(image),
        'AHE': adaptive_histogram_equalization(image, (16, 16)),
        'CLAHE (CL=1.0)': clahe_manual(image, (16, 16), 1.0)[0],
        'CLAHE (CL=2.0)': clahe_manual(image, (16, 16), 2.0)[0],
        'CLAHE (CL=4.0)': clahe_manual(image, (16, 16), 4.0)[0]
    }
    
    # Calculate quality metrics
    def calculate_metrics(img):
        # Contrast (standard deviation)
        contrast = np.std(img)
        
        # Local contrast (average local standard deviation)
        local_std = filters.rank.standard(img, disk(3))
        local_contrast = np.mean(local_std)
        
        # Edge content (using Sobel)
        sobel_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
        edge_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
        edge_content = np.mean(edge_magnitude)
        
        # Entropy
        hist, _ = np.histogram(img.flatten(), bins=256, range=[0, 256])
        hist = hist / np.sum(hist)
        entropy = -np.sum(hist[hist > 0] * np.log2(hist[hist > 0]))
        
        return {
            'contrast': contrast,
            'local_contrast': local_contrast,
            'edge_content': edge_content,
            'entropy': entropy
        }
    
    # Calculate metrics for all methods
    metrics = {}
    for name, img in methods.items():
        metrics[name] = calculate_metrics(img)
    
    # Create visualization
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Image Enhancement Quality Comparison', fontsize=16, fontweight='bold')
    
    # Display images
    for idx, (name, img) in enumerate(methods.items()):
        if idx < 6:  # Only show first 6
            row = idx // 3
            col = idx % 3
            axes[row, col].imshow(img, cmap='gray', vmin=0, vmax=255)
            axes[row, col].set_title(name)
            axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print metrics comparison
    print("📊 Image Quality Metrics Comparison:")
    print("=" * 90)
    print(f"{'Method':<15} | {'Contrast':<10} | {'Local Cont.':<12} | {'Edge Cont.':<12} | {'Entropy':<8}")
    print("-" * 90)
    
    for name, metric in metrics.items():
        print(f"{name:<15} | {metric['contrast']:<10.2f} | {metric['local_contrast']:<12.2f} | "
              f"{metric['edge_content']:<12.2f} | {metric['entropy']:<8.2f}")
    
    return methods, metrics

# Test CLAHE implementation
print("🔧 Testing CLAHE Implementation...")

# Use our challenging image
test_clahe_result, transform_funcs = clahe_manual(challenging_image, tile_size=(16, 16), clip_limit=2.0)

# Visualize clip limit effects
print("\\n📊 Analyzing Clip Limit Effects...")
visualize_clip_limit_effects(challenging_image)

# Compare enhancement quality
print("\\n🔍 Comprehensive Quality Analysis...")
enhanced_methods, quality_metrics = compare_enhancement_quality(challenging_image)

print("\\n💡 CLAHE Key Benefits:")
print("• Prevents over-enhancement through clip limiting")
print("• Maintains local contrast adaptation")
print("• Reduces noise amplification")
print("• Produces more natural-looking results")
print("• Widely used in medical imaging and photography")


In [ ]:
# Color image histogram equalization
def color_histogram_equalization(image_rgb):
    """
    Apply histogram equalization to color images using different methods
    """
    
    # Method 1: Channel-wise RGB equalization
    rgb_eq = np.zeros_like(image_rgb)
    for i in range(3):
        rgb_eq[:, :, i] = cv2.equalizeHist(image_rgb[:, :, i])
    
    # Method 2: HSV Value channel equalization
    hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)
    hsv_eq = hsv.copy()
    hsv_eq[:, :, 2] = cv2.equalizeHist(hsv[:, :, 2])  # Equalize V channel
    hsv_eq_rgb = cv2.cvtColor(hsv_eq, cv2.COLOR_HSV2RGB)
    
    # Method 3: LAB Lightness channel equalization
    lab = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2LAB)
    lab_eq = lab.copy()
    lab_eq[:, :, 0] = cv2.equalizeHist(lab[:, :, 0])  # Equalize L channel
    lab_eq_rgb = cv2.cvtColor(lab_eq, cv2.COLOR_LAB2RGB)
    
    # Method 4: YUV Luminance channel equalization
    yuv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2YUV)
    yuv_eq = yuv.copy()
    yuv_eq[:, :, 0] = cv2.equalizeHist(yuv[:, :, 0])  # Equalize Y channel
    yuv_eq_rgb = cv2.cvtColor(yuv_eq, cv2.COLOR_YUV2RGB)
    
    return {
        'RGB': rgb_eq,
        'HSV': hsv_eq_rgb,
        'LAB': lab_eq_rgb,
        'YUV': yuv_eq_rgb
    }

def color_clahe_application(image_rgb, clip_limit=2.0, tile_grid_size=(8, 8)):
    """Apply CLAHE to color images"""
    
    # Convert to LAB for better color preservation
    lab = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2LAB)
    
    # Create CLAHE object
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    
    # Apply CLAHE to L channel
    lab_clahe = lab.copy()
    lab_clahe[:, :, 0] = clahe.apply(lab[:, :, 0])
    
    # Convert back to RGB
    rgb_clahe = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2RGB)
    
    return rgb_clahe

# Load a sample color image
def create_color_test_image():
    """Create a color test image with varying lighting"""
    h, w = 300, 400
    x, y = np.meshgrid(np.linspace(0, 1, w), np.linspace(0, 1, h))
    
    # Create RGB channels with different characteristics
    r_channel = 50 + 150 * x + 30 * np.sin(10 * x) * np.sin(10 * y)
    g_channel = 100 + 100 * y + 40 * np.sin(8 * x) * np.sin(8 * y)
    b_channel = 80 + 120 * (x + y) / 2 + 20 * np.sin(12 * x) * np.sin(12 * y)
    
    # Add some darker regions
    mask = (x - 0.3)**2 + (y - 0.7)**2 < 0.1
    r_channel[mask] *= 0.3
    g_channel[mask] *= 0.3
    b_channel[mask] *= 0.3
    
    # Combine channels
    color_image = np.stack([r_channel, g_channel, b_channel], axis=2)
    color_image = np.clip(color_image, 0, 255).astype(np.uint8)
    
    return color_image

# Create test color image
test_color_image = create_color_test_image()

# Apply different color equalization methods
print("🎨 Processing Color Image Histogram Equalization...")
color_results = color_histogram_equalization(test_color_image)

# Apply color CLAHE
color_clahe_result = color_clahe_application(test_color_image)

# Visualize results
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Color Image Histogram Equalization Methods', fontsize=16, fontweight='bold')

# Original and methods
images_to_show = [
    ('Original', test_color_image),
    ('RGB Channels', color_results['RGB']),
    ('HSV (V-channel)', color_results['HSV']),
    ('LAB (L-channel)', color_results['LAB']),
    ('YUV (Y-channel)', color_results['YUV']),
    ('LAB + CLAHE', color_clahe_result)
]

for idx, (title, img) in enumerate(images_to_show):
    row = idx // 3
    col = idx % 3
    axes[row, col].imshow(img)
    axes[row, col].set_title(title)
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

# Analyze color histograms
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('RGB Channel Histograms Comparison', fontsize=16, fontweight='bold')

methods_for_hist = [
    ('Original', test_color_image),
    ('RGB Equalized', color_results['RGB']),
    ('LAB Equalized', color_results['LAB']),
    ('HSV Equalized', color_results['HSV']),
    ('YUV Equalized', color_results['YUV']),
    ('LAB + CLAHE', color_clahe_result)
]

colors = ['red', 'green', 'blue']
for idx, (title, img) in enumerate(methods_for_hist):
    row = idx // 3
    col = idx % 3
    
    for channel in range(3):
        hist, bins = np.histogram(img[:, :, channel].flatten(), bins=256, range=[0, 256])
        axes[row, col].plot(bins[:-1], hist, color=colors[channel], alpha=0.7, 
                           label=f'{colors[channel].upper()} channel')
    
    axes[row, col].set_title(title)
    axes[row, col].set_xlabel('Intensity')
    axes[row, col].set_ylabel('Frequency')
    axes[row, col].legend()
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\\n🎯 Color Enhancement Analysis:")
print("• RGB channel-wise: May cause color shifts")
print("• HSV V-channel: Preserves hue and saturation")
print("• LAB L-channel: Best perceptual color preservation")
print("• YUV Y-channel: Good for video applications")
print("• LAB + CLAHE: Optimal balance of enhancement and color fidelity")

# Performance comparison
import time

def benchmark_methods():
    """Benchmark different histogram equalization methods"""
    methods = {
        'Global HE': lambda img: cv2.equalizeHist(img),
        'Manual AHE': lambda img: adaptive_histogram_equalization(img, (16, 16)),
        'Manual CLAHE': lambda img: clahe_manual(img, (16, 16), 2.0)[0],
        'OpenCV CLAHE': lambda img: cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16)).apply(img)
    }
    
    test_img = sample_images['Low Contrast']
    
    print("\\n⚡ Performance Benchmark:")
    print("=" * 50)
    print(f"{'Method':<15} | {'Time (ms)':<10} | {'Speedup':<8}")
    print("-" * 50)
    
    times = {}
    for name, method in methods.items():
        start_time = time.time()
        for _ in range(10):  # Run 10 times for better measurement
            result = method(test_img)
        end_time = time.time()
        avg_time = (end_time - start_time) * 100  # Convert to ms
        times[name] = avg_time
    
    baseline_time = times['Global HE']
    for name, avg_time in times.items():
        speedup = baseline_time / avg_time
        print(f"{name:<15} | {avg_time:<10.2f} | {speedup:<8.2f}x")

benchmark_methods()


In [ ]:
# Essential imports for histogram equalization
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import data, exposure, filters, img_as_ubyte, img_as_float
from scipy import stats, ndimage
import seaborn as sns
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🚀 Ready to master histogram equalization!")
print(f"OpenCV version: {cv2.__version__}")

# Set random seed for reproducibility
np.random.seed(42)
